In [34]:
from sage.rings.polynomial.cyclotomic import cyclotomic_coeffs
#sampling https://doc.sagemath.org/html/en/reference/stats/sage/stats/distributions/discrete_gaussian_polynomial.html
from sage.stats.distributions.discrete_gaussian_polynomial import DiscreteGaussianDistributionPolynomialSampler

def decode(m_1, q):
    decoded = []
    
    upper_bnd = math.floor((3*q)/4)
    lower_bnd = math.floor(q/4)
    
    for element in m_1:
        if (element > lower_bnd) and (element < upper_bnd):
            decoded.append(1)
        else:
            decoded.append(0)
    return(decoded)

def gaussian_sampler_trunc(n, sigma, truncation):
    #this returns the truncated Guassian Samples
    samples = {}
    check_sign = 1
    count = 0
    gs = DiscreteGaussianDistributionPolynomialSampler(ZZ['x'], 1, sigma)

    for num in range(n): 
        error = int(gs()) 
        if(error<0):
            check_sign = -1
        else:
            check_sign = 1         
        if (truncation ==0):
            error_trunc = error
        elif (truncation == 1):
            error_trunc = abs(error) & 0b01111 #5 bits to 4 bits - max val 15
        elif (truncation == 2):
            error_trunc = abs(error) & 0b00111 #5 bits to 3 bits - max val 7
        elif (truncation == 3):
            error_trunc = abs(error) & 0b00011 #5 bits to 2 bits - max val 3
        else:
            return   
        samples[num] = error_trunc*check_sign
        
    Z.<x> = ZZ['x']
    poly_samples =  Z(samples)
        
    return poly_samples

def reg_cryptosystem_no_trunc(n,q,sigma):
    #https://gist.github.com/toyofuku/f85d57624132a44dbba1169bccf4d812
    ## this creates the ring 
    S.<x> = PolynomialRing(IntegerModRing(q),'x')
    f = x^n+1
    
    ## creates Gaussian Sampler
    gs = DiscreteGaussianDistributionPolynomialSampler(ZZ['x'], n, sigma)
    
    ### keygen ###
    #Guassian sampling
    r1, r2 = [gs() for _ in range(2)]
    
    #public keys
    a = S(ZZ[x].random_element(degree=q)).quo_rem(f)[1] ##attempt to create a random number polynomial https://ask.sagemath.org/question/24921/generating-random-polynomials-in-ri/
    p = (r1-a*r2).quo_rem(f)[1]
    
    ### encryption ###

    #Guassian sampling
    e1, e2, e3 = [gs() for _ in range(3)]
    
    #create message  
    m = {}
    for num in range(n):
        m[num] = 0
    m1 = S(m) #0

    c1a = (a*e1).quo_rem(f)[1]
    c1b = (c1a+e2).quo_rem(f)[1]
    c1 = (a*e1 + e2).quo_rem(f)[1]
    c2 = (p*e1 + e3 + m1).quo_rem(f)[1]

    ## decryption ##
    m_1 = (c1*r2+c2).quo_rem(f)[1]
    return decode(m_1, q)

def reg_cryptosystem_trunc(n,q,sigma, truncation):
    ## this creates the ring 
    S.<x> = PolynomialRing(IntegerModRing(q),'x')
    f = x^n+1
    
    ## creates Gaussian Sampler
    gs = DiscreteGaussianDistributionPolynomialSampler(ZZ['x'], n+1, sigma)
    
    ### keygen ###
    #Guassian sampling
    r1 = gaussian_sampler_trunc(n, sigma, truncation)
    r2 = gaussian_sampler_trunc(n, sigma, truncation)
    
    #public keys
    a = S(ZZ[x].random_element(degree=q)).quo_rem(f)[1] ##attempt to create a random number polynomial https://ask.sagemath.org/question/24921/generating-random-polynomials-in-ri/
    p = (r1-a*r2).quo_rem(f)[1]
    
    ### encryption ###

    #Guassian sampling
    e1 = gaussian_sampler_trunc(n, sigma, truncation)
    e2 = gaussian_sampler_trunc(n, sigma, truncation)
    e3 = gaussian_sampler_trunc(n, sigma, truncation)

    #create message  
    m = {}
    for num in range(n):
        m[num] = 0
    m1 = S(m) #0

    c1a = (a*e1).quo_rem(f)[1]
    c1b = (c1a+e2).quo_rem(f)[1]
    c1 = (a*e1 + e2).quo_rem(f)[1]
    c2 = (p*e1 + e3 + m1).quo_rem(f)[1]

    ## decryption ##
    m_1 = (c1*r2+c2).quo_rem(f)[1]
    return decode(m_1, q)

def simulation(n, q, sigma, truncation, no):
    expected_output = [0]*n
    check = 0
    check_t = 0
    
    for x in range(no):
        y = reg_cryptosystem_no_trunc(n, q, sigma)
        z = reg_cryptosystem_trunc(n,q,sigma, truncation)
        
        if (y != expected_output):
            check = check+1
        
        if (z != expected_output):
            check_t = check_t+1
    
    print('Error Percentage for non-trunc: '+str(float(check/no)))
    print('Error Percentage for trunc: '+str(float(check_t/no)))
    return check, check_t

In [35]:
simulation(256, 4093, 3.33, 1, 1000

Error Percentage for non-trunc: 0.012
Error Percentage for trunc: 0.015


(12, 15)